## Alojamientos

Dada una url de una página de booking, la inspecciona y extrae las caracteriísticas más relevantes del establecimiento.

· Nombre.

· Coordenadas (latitud y longitud).

· Dirección completa.

· Localidad.

· Capacidad de las habitaciones (lista con las opciones (1,2,3,...)).

· Precio de las habitaciones (lista con los distintos precios en el mismo orden que en la lista de capacidad de las habitaciones).

· Descripción.

· Servicios (piscina, wifi, parking, ...) (en formato texto, cada hotel tiene unos campos).

· Valoración: valoración sobre 10 que proviene de las opiniones.

· Número de votos: número de opiniones registradas.

· Noches mínimas: noches mínimas requeridas en la reserva.

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
import re
import datetime
import random

import requests
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException


import warnings
warnings.filterwarnings("ignore")

# 1. Funciones

In [5]:
def scroll_and_load_more(driver, sleep_time=2):
    """
    Función para hacer scroll hacia abajo y hacer clic en 'Cargar más resultados' si aparece.
    
    Parámetros:
    - driver: driver inicializado.
    - sleep_time: Tiempo en segundos entre cada scroll y cada intento de hacer clic en el botón "Cargar más resultados".
    """

    try:
        while True:
            # Hacer scroll hacia abajo (simulando un 'scroll' de página)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(sleep_time)  # Esperar para que la página cargue más contenido

            # Intentar encontrar el botón "Cargar más resultados"
            try:
                cargar_button = driver.find_element(By.XPATH, "//button[.//span[contains(text(), 'Cargar más resultados')]]")

                

                # driver.execute_script("window.scrollBy(0, -400);")  # Desplaza la página 200px hacia arriba
                # Si lo encontramos, hacer clic en el botón
                cargar_button.click()
                # print("Botón 'Cargar más resultados' clickeado.")
                time.sleep(sleep_time)  # Esperar un poco para que se carguen más resultados

            except NoSuchElementException:
                # Si no encontramos el botón, terminar el scroll
                # print("No se encontró el botón 'Cargar más resultados'. Finalizando.")
                break


    except:
        print("Error al cargar más resultados")

In [6]:
def get_links(checkin, checkout, n_adults, n_children, n_rooms, hotels):
    """
    Función para obtener las urls de booking correspondientes a unas características determinadas.

    Parámetros:
    - checkin: fecha de entrada al alojamiento.
    - checkout: fecha de salida del alojamiento.
    - n_adults, n_children: número de personas para los que se busca alojamiento, separadas en adultos y niños
    - hotels: lista con el nombre o nombres de los hoteles en el formato de url.

    Salida:
    - urls: lista con las urls completas para unas condiciones dadas.
    """
    urls = []
    for hotel in hotels:
        url0 = f"https://www.booking.com/hotel/es/{hotel}.es.html?label=gog235jc-1DCAEoggI46AdIClgDaEaIAQGYAQq4ARfIAQzYAQPoAQH4AQKIAgGoAgO4Ao-47rwGwAIB0gIkNmE2NTY0NGQtMjFhMy00MmE0LWEzYzAtOTk0ZTAxOTU4YzNi2AIE4AIB&aid=397594&ucfs=1&arphpl=1&checkin={checkin}&checkout={checkout}&group_adults={n_adults}&req_adults={n_adults}&no_rooms={n_rooms}&group_children={n_children}&req_children={n_children}&hpos=1&hapos=1&sr_order=popularity&srpvid=f0df6fa37fbe0907&srepoch=1738684513&all_sr_blocks=506022102_388796349_0_33_0&highlighted_blocks=506022102_388796349_0_33_0&matching_block_id=506022102_388796349_0_33_0&sr_pri_blocks=506022102_388796349_0_33_0__6500&from=searchresults"
        urls.append(url0)

    return urls

In [7]:
def detectar_tipo_alerta(texto):
    """
    Función para detectar si el alojamiento considerado con las características y fechas elegidas no está disponible o requiere de estancia mínima.

    Parámetros:
    - texto: texto referente a una alerta disponible en la página

    Salida:
    - 0 si no hay disponibilidad
    - noches: número de noches mínimas que es necesario reservar.
    """

    # Patrón para la frase de disponibilidad
    patron_disponibilidad = r"no tenemos disponibilidad"
    # Patrón para la frase de estancia mínima
    patron_estancia = r"Tienes que alojarte (\d+) noches o más"
    
    # Buscar el patrón de disponibilidad
    if re.search(patron_disponibilidad, texto, re.IGNORECASE):
        return pd.to_numeric(0)
    
    # Buscar el patrón de estancia mínima
    match_estancia = re.search(patron_estancia, texto)
    if match_estancia:
        noches = match_estancia.group(1)  # Extraer el número de noches
        return pd.to_numeric(noches)
    
    return 

In [8]:
def fechas_aleatorias(n=2):
    """
    Función para generar nuevas fechas de checkin y check out, desde la fecha actual hasta el final del año en curso.

    Parámetros:
    - n: número de noches entre checkin y checkout (por defecto 2)

    Salida:
    - fecha de checkin
    - fecha de checkout considerando una noche
    - fecha de checkout considerando n noches
    """
    hoy = datetime.date.today()
    inicio = hoy.toordinal()
    fin = datetime.date(hoy.year, 12, 31).toordinal()
    fecha_random = datetime.date.fromordinal(random.randint(inicio, fin))
    
    return (
        fecha_random.strftime("%Y-%m-%d"),
        (fecha_random + datetime.timedelta(days=1)).strftime("%Y-%m-%d"),
        (fecha_random + datetime.timedelta(days=int(n))).strftime("%Y-%m-%d")
    )

In [9]:
def extract_accommodation_data(response, print_ = False):
    """
    Función para obtener la información relevante de un determinado alojamiento a partir de una URL de Booking.com
    
    Parámetros:
    - response: objeto que contiene el html de la página cargada de Booking.com.

    Salida:
    - nombre: nombre del alojamiento.
    - lat, lng: coordenadas del alojamiento.
    - direccion: dirección completa.
    - localidad: ciudad/pueblo donde se encuentra.
    - tipo_habitacion
    - tipo_cama
    - caracteristicas
    - n_pers: lista con la capacidad de las habitaciones ofertadas.
    - precios: lista con los precios de las habitaciones.
    - texto_descripcion: descripción breve del alojamiento.
    - servicios_lista: lista con los principales servicios disponibles.
    """
    
    
    soup = BeautifulSoup(response.text, "html.parser")

    # Detectar si existe alguna alerta que impida obtener la totalidad de los datos
    alerta = soup.find(class_ = "bui-alert__title")
    if alerta: # Si hay alerta se considera como estado el código de la alerta (0 si no hay disponibilidad o noches de estancia mínima)
        texto = alerta.get_text()
        estado = detectar_tipo_alerta(texto)
    else:  # Si todo va bien 1
        estado = 1

    ############################## NOMBRE ##############################
    nombre =  soup.find(class_='ddb12f4f86 pp-header__title').get_text()
    
    ########################### COORDENADAS ############################
    try:
        coordenadas = soup.find("a", {"data-atlas-latlng": True})
        lat_lng = coordenadas["data-atlas-latlng"].split(",")
        lat, lng = lat_lng[0], lat_lng[1]
    except:
        return "KO", "lat", "lng", "direccion", "localidad","tipo_habitacion", "tipo_cama", "caracteristicas","n_pers", "precios" , "text_lists","texto_descripcion" , "servicios_lista" 
    
    ###################### LOCALIDAD Y DIRECCIÓN #######################
    try:
        direccion_tag = soup.find("div", class_="b99b6ef58f cb4b7a25d9")
        direccion = direccion_tag.contents[0].strip()
        match = re.search(r',\s*\d{5}\s+([\w\s\-]+),\s+España', direccion)
        localidad = match.group(1).strip()
    except:
        direccion = "Desconocido"
        localidad = "Desconocido"
    
    
    ########################### DESCRIPCIÓN ############################
    try:
        texto_descripcion = soup.find("p", {"data-testid": "property-description"}).get_text(strip=True)
    except:
        texto_descripcion = "Desconocido"
    
    ############################ SERVICIOS #############################
    servicios_lista = []
    try:
        servicios_completos = soup.find(class_= "e9f7361569 eb3a456445 b049f18dec").find_all(class_= "b0bf4dc58f b2f588b43c b14a6d9e99 ee817a4ef2")
        for servicio in servicios_completos:
            servicios_lista.append(servicio.get_text())
    except:
        servicios_lista =  ["Desconocido"] 
    
    ####################################################################
    # Mostrar los resultados o no
    if print_==True:
        for item in [nombre, lat, lng, direccion, localidad,texto_descripcion , servicios_lista]:
            print(item)

    return  estado, nombre, lat, lng, direccion, localidad, texto_descripcion , servicios_lista 

In [10]:
def extract_prices(response):
    """
    Función para obtener la información relevante de un determinado alojamiento a partir de una URL de Booking.com
    
    Parámetros:
    - response: objeto que contiene el html de la página cargada de Booking.com.

    Salida:
    - nombre: nombre del alojamiento.
    - lat, lng: coordenadas del alojamiento.
    - direccion: dirección completa.
    - localidad: ciudad/pueblo donde se encuentra.
    - tipo_habitacion
    - tipo_cama
    - caracteristicas
    - n_pers: lista con la capacidad de las habitaciones ofertadas.
    - precios: lista con los precios de las habitaciones.
    - texto_descripcion: descripción breve del alojamiento.
    - servicios_lista: lista con los principales servicios disponibles.
    """
    
    
    soup = BeautifulSoup(response.text, "html.parser")

    
    # #################### CARCATERÍSTICAS HABITACIÓN ####################
    # # Nombre de la habitación
    # habitaciones = [span.get_text(strip=True) for span in soup.find_all("span", class_="hprt-roomtype-icon-link")]
    # 
    # # Distribución de camas/ tipo de camas de la habitación
    # camas = [span.get_text(strip=True) for span in soup.find_all("li", class_="rt-bed-type")] # (esto igual es mejor quitarlo, la información no está unificada)
    # if not camas:
    #     camas = [span.get_text(strip=True) for span in soup.find_all("li", class_="bedroom_bed_type")]
# 
    # 
    # # Características generales 
    # caract = []
    # bloque = soup.find_all('div', class_='hprt-facilities-block')
    # for bloq in bloque:
    #     try:
    #         facilities = bloq.find_all('div', class_='hprt-facilities-facility')
    #         # Extraer los textos de los elementos <span>
    #         facilities_text = [facility.find('span').get_text(strip=True) for facility in facilities if facility.find('span')]
    #         if len(facilities_text) > 0:
    #             caract.append(facilities_text)
    #         # print(facilities_text)
    #     except:
    #         print(f"No funciona")
    
    # # Obtener el número de ofertas con cada habitación, tipo de cama y caracteristicas
    # filas  = [td_element['rowspan'] for td_element in soup.find_all('td', {'rowspan': True})]
    # filas = pd.to_numeric(filas)
# 
    # if len(camas) < len(filas):
    #     camas = ["Desconocido"] * len(filas)
# 
    # if len(habitaciones) < len(filas):
    #     habitaciones = ["Desconocido"] * len(filas)
    # 
    # if len(caract) < len(filas):
    #     caract = ["Desconocido"] * len(filas)
    # 
    # 
    # # Crear listas de habitación, tipo de cama y características acorde al número de veces que se repite
    # tipo_habitacion, tipo_cama, caracteristicas = [], [], [] # Inicializamos las nuevas listas vacías
    # for l1, l2, l3, count in zip(habitaciones, camas, caract, filas):
    #     tipo_habitacion.extend([l1] * count)  # Repetimos l1 el número de veces indicado en count
    #     tipo_cama.extend([l2] * count)  # Repetimos l2 el número de veces indicado en count
    #     caracteristicas.extend([l3] * count)  # Repetimos l3 el número de veces indicado en count
   
    ####################### CAPACIDAD HABITACIÓN #######################
    # Obtener la capacidad y el precio de cada habitación
    n_pers = []
    # Buscar capacidades de habitaciones
    capacidades = soup.find_all(class_="bui-u-sr-only")
    for capacidad in capacidades:
        text = capacidad.get_text(strip=True)
        match = re.search(r'Máximo de personas: (\d+)', text)
        if match:
            n_pers.append(int(match.group(1)))
    
    ######################## PRECIO HABITACIÓN #########################
    precios = []
    precios_act = soup.find_all(class_="hprt-price-block")
    for casilla in precios_act:
        precios_elementos = casilla.find_all(class_="prco-valign-middle-helper")
        for precio in precios_elementos:
            precios.append(int(precio.get_text(strip=True).replace('€', '').replace('.', '').strip()))
    
    ##################### CARACTERÍSTICAS RESERVA ######################
    li_elements = soup.find_all(class_='hprt-conditions-bui bui-list bui-list--text bui-list--icon bui-f-font-caption')
    text_lists = []
    for elemento in li_elements:
        lista_ind = []
        # Aquí buscamos todos los elementos <li> dentro de este bloque
        texto = elemento.find_all(class_="bui-list__item")
        for elemento in texto:
            lista_ind.append(elemento.get_text(strip=False).replace('\n', ' ').strip() )  # Extraemos el texto limpio
        text_lists.append(lista_ind)

    

    
    ####################################################################


    return  n_pers, precios , text_lists # tipo_habitacion, tipo_cama, caracteristicas,

In [12]:
meses = {
    "ene": 1, "feb": 2, "mar": 3, "abr": 4, "may": 5, "jun": 6,
    "jul": 7, "ago": 8, "sep": 9, "oct": 10, "nov": 11, "dic": 12
}
def convertir_fecha(fecha_str):
    """
    Función para transformar un string en una fecha de inicio y una fecha final

    Parámetros:
    - fecha_str: string con fechas

    Salida:
    - fecha_inicio: primera fecha de la cadena
    - fecha_fin: segunda fecha de la cadena
    """
    # Partir la fecha en dos partes
    fecha_inicio, fecha_fin = fecha_str.split(" – ")

    # Función interna para convertir una fecha
    def convertir(fecha):
        """
        Función para transformar a fechas cadenas de texto en formato 00-mes. 

        Parámetros:
        - fecha: fecha en formato string (dd(numero) - mmm(string))

        Salida:
        - fecha en formato string con el formato adecuado (YYYY-mm-dd)
        """
        dia, mes_abrev = fecha.split(" ")
        mes = meses[mes_abrev.lower()]  # Convertir el mes abreviado a número
        año = datetime.datetime.now().year  # Obtener el año actual
        # Devolver la fecha en formato "YYYY-MM-DD"
        return f"{año}-{mes:02d}-{int(dia):02d}"

    # Convertir ambas fechas
    fecha_inicio = convertir(fecha_inicio)
    fecha_fin = convertir(fecha_fin)

    return fecha_inicio, fecha_fin

# 2. Localizar los alojamientos

Considerando las localidades más importantes o turísticas, se busca extraer de la página de booking que contiene todos los alojamientos para cada una de ellas, las urls de esos alojamientos. En concreto, nos interesa extraer el nombre del alojamiento en la url, para poder acceder a sus características en el momento y en las condiciones necesarias.

Para las localidades con menos de 25 alojamientos, se extraen directamente con "requests" (se extraen 25, porque muestra también alojamientos cercanos a la localidad y no supone un tiempo extra). Para las localidades con más de 25 alojamientos, se utiliza "Selenium" para hacer scroll y cargar todos los resultados, y luego se extrae el nombre con "requests".

Además del nombre en la url, se extraen la valoración media y el número de votos de cada alojamiento.

In [56]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

# Determinar las localidades que quueremos tener en cuenta
ciudades = ["Allande", "Aller", "Arriondas", "Aviles", "Belmonte de Miranda", "Bimenes", "Boal", "Cabrales", "Candamo","Candás", "Cangas de Onis", "Cangas del Narcea", 
            "Castropol", "Coaña", "Colunga", "Cudillero", "El Franco", "Gijon", "Grado Asturias", "Grandas de Salime", "Langreo", "Lastres", "Laviana", "Llanera", 
            "Llanes", "Luanco", "Luarca", "Lugones", "Mieres", "Muros de Nalon", "Nava Asturias", "Navia", "Noreña", "Oviedo", "Pilona", "Pola de Lena", "Pravia", "Puerto de Vega",
            "Ribadesella", "Salas", "Salinas Asturias", "Siero", "Somiedo", "Soto del Barco", "Tapia", "Taramundi", "Tineo", "Vegadeo", "Villaviciosa"]
distancia = "5000" 

# Almacenar resultados
nombres_urls = []
valoraciones = []
numero_votos = []
localidad = []

# Contador del total de alojamientos (debería ser menor que los que obtendremos)
total = 0


for id in range(len(ciudades)): # Recorremos el vector de ciudades
    # Determinar la ciudad seleccionada y extraer el html
    ciudad = ciudades[id]
    ciudad = ciudad.replace(" ", "+")
    url = f"https://www.booking.com/searchresults.es.html?aid=304142order=price&ss=+{ciudad}+&nflt={distancia}%3D+{distancia}+&offset=1"
    response = requests.get(url, headers=headers) # Hacer la solicitud a la página
    soup = BeautifulSoup(response.text, "html.parser") # Extraer el html

    # Extraer el número de alojamientos de los que dispone
    n_alojamientos = pd.to_numeric(soup.find(class_ = "b87c397a13 cacb5ff522").get_text().split(":")[1].split(" ")[1])
    total += n_alojamientos

    # En caso de tener que cargar más resultados, utilizar Seleniun para cargar la página completa
    if n_alojamientos > 25:
        # Configurar Selenium y cargar la página
        options = Options() 
        # options.add_argument('--headless')  # Para ejecutar sin abrir el navegador
        driver = webdriver.Chrome(options=options)
        driver.get(url)
        time.sleep(2)

        wait = WebDriverWait(driver, 10) # Espera para cargar la página
        
        # Aceptamos las cookies
        boton_aceptar = wait.until(
            EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
        )
        boton_aceptar.click() 

        # Hacer scroll y cargar todos los alojamientos
        scroll_and_load_more(driver, sleep_time=2)
        time.sleep(2)

        # Extraer el HTML después del scroll
        html = driver.page_source
        driver.quit()  # Cerrar el driver
    
        soup = BeautifulSoup(html, "html.parser")

    # Localizar todos los alojamientos de la página
    enlace = soup.find_all("a", {"data-testid": "review-score-link"})
    
    # Para cada alojamiento extraer la url, la valoracion media y el numero de votos
    for i in enlace:
        url = i["href"]  # Extraer la URL
        nombres_urls.append(url[33:].split(".es")[0])
        localidad.append(ciudades[id])

        try:
            valoraciones.append(i.find(class_ = "bc946a29db").get_text().split(" ")[1].replace(",", "."))
        except:
            valoraciones.append("Desconocido")
        
        try:
            numero_votos.append(i.find(class_ = "fff1944c52 fb14de7f14 eaa8455879").get_text().split(" ")[0].replace(".", ""))
        except:
            numero_votos.append("Desconocido")
        

        
print(total)
# Crear un dataframe conjunnto
nombres_alojamientos = pd.DataFrame({"nombre_url": nombres_urls,
                                     "valoraciones": valoraciones,
                                     "numero_votos": numero_votos,
                                     "localidad": localidad})

3491


In [ ]:
# Eliminamos los duplicados, por si se han solapado algunas zonas (si tienen el mismo nombre en la url, nos daran el mismo alojamiento despues)
nombres_alojamientos.drop_duplicates(subset = "nombre_url", inplace=True)
nombres_alojamientos.shape

(2647, 4)

In [60]:
# Generamos urls para unas condiciones determinadas.
checkin, checkout, _ = fechas_aleatorias(n=2)
n_adults = 2
n_rooms = 1
n_children = 0

print("Fecha de checkin considerada incialmente: ", checkin)
print("Fecha de checkout considerada incialmente: ", checkout)

# Añadimos las urls con las nuevas condiciones y el estado (lo inicializamos el 0 por defecto) como nuevas columnas del dataframe con los datos
nombres_alojamientos["urls"]= get_links(checkin, checkout, n_adults, n_children, n_rooms, nombres_alojamientos["nombre_url"])
nombres_alojamientos["estado"] = 0 # Realmente no nos hace falta

Fecha de checkin considerada incialmente:  2025-06-18
Fecha de checkout considerada incialmente:  2025-06-19


In [ ]:
# Guardamos los datos
# nombres_alojamientos.to_csv("../../Data/Data_used/accommodation_names_urls.csv", index=False)

# 3. Extraer caracterísiticas de los datos

Para cada alojamiento se extraen las caracteristicas fijas, aquellas que no dependen de la disponibilidad. Además, se extrae la variable que determina el estado (1:disponible ; 0:no disponible; >1: requiere un número de noches mínimo)

In [ ]:
# nombres_alojamientos = pd.read_csv("../../Data/Data_used/accommodation_names_urls.csv")
nombres_alojamientos.head()

,nombre_url,valoraciones,numero_votos,localidad,urls,estado
0,nueva-allandesa,8.0,1297,Allande,https://www.booking.com/hotel/es/nueva-allande...,0
1,casa-gaya3n,9.3,144,Allande,https://www.booking.com/hotel/es/casa-gaya3n.e...,0
2,apartamentos-rurales-casa-carmen,9.5,82,Allande,https://www.booking.com/hotel/es/apartamentos-...,0
3,casa-el-rey-12-apartamentos-buenos-aires-y-fig...,9.3,55,Allande,https://www.booking.com/hotel/es/casa-el-rey-1...,0
4,albergue-los-hospitales,9.3,389,Allande,https://www.booking.com/hotel/es/albergue-los-...,0


In [86]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
# Condiciones iniciales generales
n_adults = 2
n_rooms = 1
n_children = 0

# Almacenar los datos
datos = []

# Recorremos los alojamientos para extraer sus caracterísiticas
for id in range(0,nombres_alojamientos.shape[0]):#  nombres_alojamientos.shape[0]
    response = requests.get(nombres_alojamientos.loc[id, "urls"], headers=headers) # Hacer la solicitud a la página
    
    # Extraer las características de la página
    estado, nombre, lat, lng, direccion, localidad, texto_descripcion, servicios_lista = extract_accommodation_data(response)

    # Añadir los datos del nuevo alojamiento
    # Si el bucle ha parado por llegar al número máximo de iteraciones, se tendran algunos campos vacíos.
    datos.append({
            'Nombre': nombre, 'Localización': localidad, 'Latitud': lat, 'Longitud': lng,
            'Dirección': direccion, # "Número de personas": n_pers, "Precio": precios, 
            # 'Tipo_habitacion': tipo_habitacion, "Tipo_cama": tipo_cama, "Carac_habitacion": caracteristicas,
            "Valoración": nombres_alojamientos.loc[id, "valoraciones"], 
            "Votos": nombres_alojamientos.loc[id, "numero_votos"], # "Noches mínimas": n_noches,
            "Servicios": servicios_lista, "Descripción": texto_descripcion, "url": nombres_alojamientos.loc[id, "urls"],
            "estado": estado, "id_nombre":  nombres_alojamientos.loc[id, "nombre_url"]
        })
    
# Crear un dataframe con los datos
df_resultado = pd.DataFrame(datos)

In [ ]:
# Añadir los nuevos datos al archivo csv
# df_resultado.to_csv("../../Data/Data_used/accommodations_features.csv", mode='a', header=not pd.io.common.file_exists("caracteristicas_alojamientos.csv"), index=False)

# 4. Tratamiento de alojamientos sin fecha

Algunos alojamientos no están disponibles en la fecha seleccionada, por lo que es necesario tratarlos. Se tienen dos casos: aquellos que requieren de un número de noches mínimas y aquellos que no están disponibles (podrían necesitar también un número de noches mínimas).

In [ ]:
# df = pd.read_csv("../../Data/Data_used/accommodations_features.csv")
df = df_resultado.copy()
print(df.shape)
df["estado"].value_counts() # Solo 594 con disponibilidad

(2647, 12)


estado
0    998
2    824
1    511
3    179
4     72
5     42
7     13
6      8
Name: count, dtype: int64

Buscar fechas para los que tienen noche mínima. Algunos sugieren en las que hay disponibilidad. Introducimos el número de noches (n_noches) mínimo que es necesario para el alojamiento.

In [93]:
df_minimo = df[df["estado"] > 1].reset_index(drop = True)
df_minimo["n_noches"] = "Desconocido"
df_minimo.head()

,Nombre,Localización,Latitud,Longitud,Dirección,Valoración,Votos,Servicios,Descripción,url,estado,id_nombre,n_noches
0,Apartamentos Rurales Casa Carmen,Pola de Allande,43.232202,-6.563130,"Puente de linares s/n, 33890 Pola de Allande, ...",9.5,82,"['WiFi gratis ', 'Parking gratis ', 'Habitacio...",Apartamentos Rurales Casa Carmen está en Pola ...,https://www.booking.com/hotel/es/apartamentos-...,2,apartamentos-rurales-casa-carmen,Desconocido
1,Casa el rey 12 - Apartamentos Buenos Aires y F...,Valbona,43.262196,-6.576329,"Casa el rey, 12 33889 Figueras Allande Asturia...",9.3,55,['Parking gratis '],Casa el rey 12 - Apartamentos Buenos Aires y F...,https://www.booking.com/hotel/es/casa-el-rey-1...,2,casa-el-rey-12-apartamentos-buenos-aires-y-fig...,Desconocido
2,Apartamentos Rurales Ca Lulón,Navelgas,43.375822,-6.557153,"Yerbo, 28, 33879 Navelgas, España",9.5,42,"['Traslado aeropuerto ', 'Parking gratis ', 'H...",Apartamentos Rurales Ca Lulón está en Navelgas...,https://www.booking.com/hotel/es/apartamentos-...,2,apartamentos-rurales-ca-lula3n,Desconocido
3,La Cabana´l Cachican,Cangas del Narcea,43.158354,-6.543884,"Cangas del Narcea Numero 2, 33817 Cangas del N...",9.6,123,"['Habitaciones sin humo ', 'Parking gratis ', ...",La Cabana´l Cachican está en Cangas del Narcea...,https://www.booking.com/hotel/es/la-cabana-l-c...,2,la-cabana-l-cachican,Desconocido
4,Apartamentos Legazpi,Cangas del Narcea,43.177694,-6.550340,"Las Huertas 1, 2do., 33800 Cangas del Narcea, ...",8.5,59,"['Habitaciones sin humo ', 'WiFi gratis ', 'Ha...",Apartamentos Legazpi tiene vistas a la montaña...,https://www.booking.com/hotel/es/apartamentos-...,2,apartamentos-legazpi-cangas-del-narcea,Desconocido


In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
# Condiciones iniciales generales
n_adults = 2
n_rooms = 1
n_children = 0

# Inicializar el driver
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

# Iterar sobre el rango de URLs (iterar por partes por si hay algun problema)
for id in range(0,df_minimo.shape[0]):
    url = df_minimo.iloc[id]["url"] # URL del alojamiento es las fechas no disponibles
    
    # Acceder a la URL (siempre en la misma pestaña)
    driver.get(url) 
    time.sleep(2)  # Esperar a que cargue la página

    # Buscar el elemento que contiene fechas disponibles.
    try:
        fecha_nueva = driver.find_element(By.CLASS_NAME, "bui-group--medium").text.split("\n")[0]
        checkin_new, checkout_new = convertir_fecha(fecha_nueva) # Adaptar las fechas al formato requerido
        nuevo_link = get_links(checkin_new, checkout_new, n_adults, n_children, n_rooms, [df_minimo.iloc[id]["id_nombre"]]) # URL de las nuevas fechas

        # Actualizar las columnas del dataframe
        df_minimo.loc[id, "url"] = nuevo_link[0]
        df_minimo.loc[id, "n_noches"] = df_minimo.iloc[id]["estado"]
        df_minimo.loc[id, "estado"] = 1

    except:
        print("No da fechas de disponibilidad: ", url)
    
# Cerrar el driver después de terminar todas las iteraciones
driver.quit()

No da fechas de disponibilidad:  https://www.booking.com/hotel/es/el-lagar-2-bajo.es.html?label=gog235jc-1DCAEoggI46AdIClgDaEaIAQGYAQq4ARfIAQzYAQPoAQH4AQKIAgGoAgO4Ao-47rwGwAIB0gIkNmE2NTY0NGQtMjFhMy00MmE0LWEzYzAtOTk0ZTAxOTU4YzNi2AIE4AIB&aid=397594&ucfs=1&arphpl=1&checkin=2025-06-18&checkout=2025-06-19&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=f0df6fa37fbe0907&srepoch=1738684513&all_sr_blocks=506022102_388796349_0_33_0&highlighted_blocks=506022102_388796349_0_33_0&matching_block_id=506022102_388796349_0_33_0&sr_pri_blocks=506022102_388796349_0_33_0__6500&from=searchresults
No da fechas de disponibilidad:  https://www.booking.com/hotel/es/amplio-y-centrico-apartamento-en-aviles-by-bebalmy.es.html?label=gog235jc-1DCAEoggI46AdIClgDaEaIAQGYAQq4ARfIAQzYAQPoAQH4AQKIAgGoAgO4Ao-47rwGwAIB0gIkNmE2NTY0NGQtMjFhMy00MmE0LWEzYzAtOTk0ZTAxOTU4YzNi2AIE4AIB&aid=397594&ucfs=1&arphpl=1&checkin=2025-06-18&checkout=2025-06-19&group_adults=2

In [ ]:
# df_minimo.to_csv("../../Data/Data_used/min_aux.csv", index = False)

Buscar fechas para los que no tienen disponibilidad en las fechas seleccionadas. Algunos sugieren en las que hay disponibilidad. Introducimos el número de noches (n_noches) mínimo que es necesario para el alojamiento.

In [97]:
df_no_disponible = df[df["estado"] == 0]
df_no_disponible.reset_index(drop = True, inplace=True)
df_no_disponible["n_noches"] = "Desconocido"
df_no_disponible.head()

,Nombre,Localización,Latitud,Longitud,Dirección,Valoración,Votos,Servicios,Descripción,url,estado,id_nombre,n_noches
0,Hotel Nueva Allandesa,Pola de Allande,43.271429,-6.609451,"Donato Fernández, 3, 33880 Pola de Allande, Es...",8.0,1297,"['WiFi gratis ', 'Habitaciones sin humo ', 'Re...",El Hotel Nueva Allandesa se encuentra en el ce...,https://www.booking.com/hotel/es/nueva-allande...,0,nueva-allandesa,Desconocido
1,Casa Gayón,Pola de Allande,43.273815,-6.619590,"El Mazo, 4, 33890 Pola de Allande, España",9.3,144,"['WiFi gratis ', 'Parking gratis ', 'Habitacio...","Casa Gayón, que cuenta con jardín, terraza y r...",https://www.booking.com/hotel/es/casa-gaya3n.e...,0,casa-gaya3n,Desconocido
2,Albergue Los Hospitales,Colinas de Arriba,43.313556,-6.592020,"Colinas de Arriba, Spain, 33878 Colinas de Arr...",9.3,389,"['Parking gratis ', 'Bar ', 'Desayuno ']","Albergue Los Hospitales, en Colinas de Arriba,...",https://www.booking.com/hotel/es/albergue-los-...,0,albergue-los-hospitales,Desconocido
3,Hotel Restaurante La Casilla,Cangas del Narcea,43.151659,-6.537744,"Limes, 67, 33800 Cangas del Narcea, España",8.1,744,"['Traslado aeropuerto ', 'Habitaciones sin hum...",El hotel y restaurante La Casilla se encuentra...,https://www.booking.com/hotel/es/restaurante-l...,0,restaurante-la-casilla,Desconocido
4,27A01 Casa Uría - Cam. Santiago,Berducedo,43.233340,-6.767230,"18 Lugar Berducedo, 33887 Berducedo, España",9.8,57,"['Parking gratis ', 'WiFi gratis ']",27A01 Casa Uría - Cam. Santiago es un alojamie...,https://www.booking.com/hotel/es/27a01-casa-ur...,0,27a01-casa-uria-cam-santiago,Desconocido


In [98]:
df_no_disponible.shape

(998, 13)

In [99]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
# Condiciones iniciales generales
n_adults = 2
n_rooms = 1
n_children = 0
# Almacenar los datos
datos = []

# Recorremos los alojamientos que no tienen disponibilidad (por partes para encontrar posibles errores)
for id in range(0, df_no_disponible.shape[0]):
    cont = 0 # Número máximo de iteraciones por alojamiento
    while df_no_disponible.loc[id, "estado"] != 1 and cont <= 3:  # Mientras no tenga disponibilidad y el número de iteraciones sea menor de 3
        cont += 1  # Contabilizar la iteración

        if df_no_disponible.loc[id, "estado"] == 0: # Si no hay disponbilidad
    
            # Obtener fechas aleatorias 
            checkin, checkout, _ = fechas_aleatorias()

            # Generar nueva URL y actualizarla en el dataframe
            nueva_url = get_links(checkin, checkout, n_adults, n_children, n_rooms, [df_no_disponible.loc[id, "id_nombre"]])[0]
            df_no_disponible.loc[id, "url"] = nueva_url  

            # Hacer la solicitud a la página y extraer el html
            response = requests.get(nueva_url, headers=headers)
            soup = BeautifulSoup(response.text, "html.parser")

            # Detectar si existe alguna alerta (no disponible / noches mínimas)
            alerta = soup.find(class_="bui-alert__title")
            if alerta:  
                # Extraer el texto de la alerta y determinar el estado
                texto = alerta.get_text()
                estado = detectar_tipo_alerta(texto)  # Función que analiza la alerta
            else:  
                estado = 1  # Si no hay alerta, hay disponibilidad
                df_no_disponible.loc[id, "n_noches"] = 1

            # Actualizar el estado en el dataFrame
            df_no_disponible.loc[id, "estado"] = estado  
            
        
        if df_no_disponible.loc[id, "estado"] > 1: # Si se requieren noches mínimas
                url = df_no_disponible.iloc[id]["url"] # Extraer la url
                
                # Inicializar el driver y cargar la página
                chrome_options = Options()
                driver = webdriver.Chrome(options=chrome_options)
                driver.get(url)  
                time.sleep(2)  # Esperar a que cargue la página

                # Buscar si contiene una fecha como sugerencia
                try:
                    fecha_nueva = driver.find_element(By.CLASS_NAME, "bui-group--medium").text.split("\n")[0]
                    checkin_new, checkout_new = convertir_fecha(fecha_nueva)
                    nuevo_link = get_links(checkin_new, checkout_new, n_adults, n_children, n_rooms, [df_no_disponible.iloc[id]["id_nombre"]])

                    # Actualizar el dataframe
                    df_no_disponible.loc[id, "url"] = nuevo_link[0]
                    df_no_disponible.loc[id, "n_noches"] = df_no_disponible.iloc[id]["estado"]
                    df_no_disponible.loc[id, "estado"] = 1

                except:
                    print("No da fechas de disponibilidad: ", url)
                
                driver.quit()

No da fechas de disponibilidad:  https://www.booking.com/hotel/es/una-asturias-especial.es.html?label=gog235jc-1DCAEoggI46AdIClgDaEaIAQGYAQq4ARfIAQzYAQPoAQH4AQKIAgGoAgO4Ao-47rwGwAIB0gIkNmE2NTY0NGQtMjFhMy00MmE0LWEzYzAtOTk0ZTAxOTU4YzNi2AIE4AIB&aid=397594&ucfs=1&arphpl=1&checkin=2025-10-02&checkout=2025-10-03&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=f0df6fa37fbe0907&srepoch=1738684513&all_sr_blocks=506022102_388796349_0_33_0&highlighted_blocks=506022102_388796349_0_33_0&matching_block_id=506022102_388796349_0_33_0&sr_pri_blocks=506022102_388796349_0_33_0__6500&from=searchresults
No da fechas de disponibilidad:  https://www.booking.com/hotel/es/una-asturias-especial.es.html?label=gog235jc-1DCAEoggI46AdIClgDaEaIAQGYAQq4ARfIAQzYAQPoAQH4AQKIAgGoAgO4Ao-47rwGwAIB0gIkNmE2NTY0NGQtMjFhMy00MmE0LWEzYzAtOTk0ZTAxOTU4YzNi2AIE4AIB&aid=397594&ucfs=1&arphpl=1&checkin=2025-10-02&checkout=2025-10-03&group_adults=2&req_adults=2&no_rooms=

In [ ]:
# df_no_disponible.to_csv("../../Data/Data_used/not_available_helper.csv", index = False)

# 5. Unir dataframes y encontrar urls faltantes

Actualizar el dataframe conjunto con las urls correspondientes a las nuevas fechas encontradas

In [ ]:
# df =pd.read_csv("../../Data/Data_used/accommodations_features.csv")
# minimo = pd.read_csv("../../Data/Data_used/min_aux.csv")
# no_disponible = pd.read_csv("../../Data/Data_used/not_available_helper.csv")

df["n_noches"] = "Desconocido" # Añadir el número de noches al dataframe conjunto
df.loc[df["estado"] == 1, "n_noches"] = 1 # Para los alojamientos con disponibilidad, el n_noches es 1
minimo = df_minimo.copy()
no_disponible = df_no_disponible.copy()
df.head()

,Nombre,Localización,Latitud,Longitud,Dirección,Valoración,Votos,Servicios,Descripción,url,estado,id_nombre,n_noches
0,Hotel Nueva Allandesa,Pola de Allande,43.271429,-6.609451,"Donato Fernández, 3, 33880 Pola de Allande, Es...",8.0,1297,"['WiFi gratis ', 'Habitaciones sin humo ', 'Re...",El Hotel Nueva Allandesa se encuentra en el ce...,https://www.booking.com/hotel/es/nueva-allande...,0,nueva-allandesa,Desconocido
1,Casa Gayón,Pola de Allande,43.273815,-6.619590,"El Mazo, 4, 33890 Pola de Allande, España",9.3,144,"['WiFi gratis ', 'Parking gratis ', 'Habitacio...","Casa Gayón, que cuenta con jardín, terraza y r...",https://www.booking.com/hotel/es/casa-gaya3n.e...,0,casa-gaya3n,Desconocido
2,Apartamentos Rurales Casa Carmen,Pola de Allande,43.232202,-6.563130,"Puente de linares s/n, 33890 Pola de Allande, ...",9.5,82,"['WiFi gratis ', 'Parking gratis ', 'Habitacio...",Apartamentos Rurales Casa Carmen está en Pola ...,https://www.booking.com/hotel/es/apartamentos-...,2,apartamentos-rurales-casa-carmen,Desconocido
3,Casa el rey 12 - Apartamentos Buenos Aires y F...,Valbona,43.262196,-6.576329,"Casa el rey, 12 33889 Figueras Allande Asturia...",9.3,55,['Parking gratis '],Casa el rey 12 - Apartamentos Buenos Aires y F...,https://www.booking.com/hotel/es/casa-el-rey-1...,2,casa-el-rey-12-apartamentos-buenos-aires-y-fig...,Desconocido
4,Albergue Los Hospitales,Colinas de Arriba,43.313556,-6.592020,"Colinas de Arriba, Spain, 33878 Colinas de Arr...",9.3,389,"['Parking gratis ', 'Bar ', 'Desayuno ']","Albergue Los Hospitales, en Colinas de Arriba,...",https://www.booking.com/hotel/es/albergue-los-...,0,albergue-los-hospitales,Desconocido


Combinar los 2 dataframes (minimo y no_disponibles) con el dataframe original, de manera que se actualicen las urls, el estado y el número de noches de aquellos hoteles para los que se han encontrado fechas con disponibilidad.

In [104]:
df_aux = df.merge(minimo[["id_nombre", "url", "estado", "n_noches"]], on="id_nombre", how="left", suffixes=("", "_nuevo"))

# Reemplazar solo los valores actualizados sin tocar los demás
df_aux["url"] = df_aux["url_nuevo"].fillna(df_aux["url"])
df_aux["estado"] = df_aux["estado_nuevo"].fillna(df_aux["estado"])
df_aux["n_noches"] = df_aux["n_noches_nuevo"].fillna(df_aux["n_noches"])

# Eliminar las columnas auxiliares
df_aux.drop(columns=[ "url_nuevo", "estado_nuevo", "n_noches_nuevo"], inplace=True)


df_aux = df_aux.merge(no_disponible[["id_nombre", "url", "estado", "n_noches"]], on="id_nombre", how="left", suffixes=("", "_nuevo"))

# Reemplazar solo los valores actualizados sin tocar los demás
df_aux["url"] = df_aux["url_nuevo"].fillna(df_aux["url"])
df_aux["estado"] = df_aux["estado_nuevo"].fillna(df_aux["estado"])
df_aux["n_noches"] = df_aux["n_noches_nuevo"].fillna(df_aux["n_noches"])

# Eliminar las columnas auxiliares
df_aux.drop(columns=[ "url_nuevo", "estado_nuevo", "n_noches_nuevo"], inplace=True)

In [105]:
df_aux["estado"].value_counts()

estado
1.0    2025
0.0     392
2.0      80
5.0      76
7.0      27
6.0      24
3.0      18
4.0       5
Name: count, dtype: int64

Guardar, por separado, los alojamientos con disponibilidad y los alojamientos sin disponibilidad (los que no tienen fechas válidas).

In [ ]:
# Alojamientos con disponibilidad
alojamientos_disponibles = df_aux[df_aux["estado"] == 1]
print(alojamientos_disponibles.shape) # 2603 alojamientos con disponibilidad
alojamientos_disponibles.to_csv("../../Data/Data_used/available_accommodation_data.csv", index=False)

(2025, 13)


In [ ]:
# Alojamientos sin disponibilidad
no_disponibles = df_aux[df_aux["estado"] != 1]
print(no_disponibles.shape) # 433 alojamientos sin disponibilidad
no_disponibles.to_csv("../../Data_used/unavailable_accommodation_data.csv", index=False)

(622, 13)


# 6. Extraer características de alojamientos con disponibilidad

Para los alojamientos con disponibilidad, se extraen el resto de características, es decir, aquellas relacionadas con los diferentes precios posibles.

In [20]:
alojamientos_disponibles = pd.read_csv("../../Data/Data_used/available_accommodation_data.csv")
alojamientos_disponibles.shape

(2302, 19)

In [ ]:
# Inicializar las variables que contendrán las nuevas características
# alojamientos_disponibles["Tipo_habitacion"] = None
# alojamientos_disponibles["Tipo_cama"] = None
# alojamientos_disponibles["Carac_habitacion"] = None
# alojamientos_disponibles["Número de personas"] = None
# alojamientos_disponibles["Precio"] = None
# alojamientos_disponibles["Carac_reserva"] = None

In [14]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
for index in range(1000, 1500): # alojamientos_disponibles.shape[0]
    url = alojamientos_disponibles.loc[index, "url"]

    # Hacer la solicitud a la página
    response = requests.get(url, headers=headers)

    # Extraer los precios con sus correspondientes características
    n_pers, precios , text_lists = extract_prices(response) # tipo_habitacion, tipo_cama, caracteristicas,
    
    # Añadir los resultados al dataframe
    alojamientos_disponibles.at[index, "Número de personas"] = n_pers
    alojamientos_disponibles.at[index, "Precio"] = precios
    # alojamientos_disponibles.at[index, "Tipo_habitacion"] = tipo_habitacion
    # alojamientos_disponibles.at[index, "Tipo_cama"] = tipo_cama
    # alojamientos_disponibles.at[index, "Carac_habitacion"] = caracteristicas
    alojamientos_disponibles.at[index, "Carac_reserva"] = text_lists

In [ ]:
# Guardar los alojamientos disponibles con sus precios y caracteristicas. (2450)
alojamientos_disponibles["Precio"] = alojamientos_disponibles["Precio"].astype(str)
alojamientos_disponibles[alojamientos_disponibles["Precio"] != "[]"].to_csv("../../Data/Data_used/available_accommodation_data.csv", index=False)

In [ ]:
# Los alojamientos no disponibles los pasamos al dataframe correspondiente (153)
no_disponibles_new = alojamientos_disponibles[alojamientos_disponibles["Precio"] == "[]"][["Nombre","Localización","Latitud","Longitud","Dirección","Valoración","Votos","Servicios","Descripción","url","estado","id_nombre","n_noches"]]
no_disponibles_new.to_csv("../../Data/Data_used/unavailable_accommodation_data.csv", mode='a', header=False, index=False)